In [2]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re
import pandas as pd
import os

In [3]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')


In [4]:
import numpy as np
from tqdm import tqdm

In [49]:
model['maman'].shape

(300,)

In [5]:
def get_definitions(word, from_lang, to_lang, print_output = False):
    senseword_list = []
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    if print_output == True: print(url)
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    #print(soup.prettify())
    first_string_passed = False
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            senseword_list.append(sence_word)
            #print(sence_word)
        first_string_passed = True
    if len (senseword_list) == 0:
        print("turn to alternative marks")
        for link in soup.find_all("div", attrs={"lang" : "fr"}):
            senseword_list.append(link.text.strip())
    time.sleep(2)
    return senseword_list
    """
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,2.9))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        print("========================================")
        print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + word + "\w{0,2}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                print(example)
    """
 
get_definitions("согревать", "русский", "французский", print_output = False)



turn to alternative marks


['réchauffer', 'chaud']

In [6]:
df_en_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_lemmas_only.csv")
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,для,"['for the bank', 'for last month', 'for one ye...","['для банка', 'за последний месяц', 'на один г..."
1,1,for,ради,['for god'],['ради бога']
2,2,for,в течение,[],[]
3,3,for,в отношении,[],[]
4,4,for,для целей,[],[]


In [7]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

In [8]:
from  sklearn.metrics.pairwise import cosine_similarity


In [ ]:
fr_list = ['mère', 'maman', 'mère de famille', 
           'materner', 'Vierge', 'Mother', 'parents', 'femme', 'maternel', 'fils', 'Dame', 'Marie']

In [9]:
def fr_lemmatize(word):
    lemma = lemmatizer.lemmatize(word)
    return lemma
def preprocess_words(words_list, lemmatizer_func):
    preprocessed_words = []
    for word in words_list:
        lemma = lemmatizer_func(word.lower())
        #print(lemma)
        preprocessed_words.append(lemma)
    return preprocessed_words
        

In [10]:
l1 = "maman"
l2 = 'mère de famille'

def get_vectors_sum(lemm_line):
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2):
    l1_vect = get_vectors_sum(line_1)
    l2_vect = get_vectors_sum(line_2)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2)      

0.3063980830674433

In [11]:
def get_two_target_lang_lists_intersection(list_1, list_2):
    overall_intersection = []
    cos_sim_dict ={}
    for word_1 in list_1:
        for word_2 in list_2:
            if word_1 == word_2:
                #overall_intersection.append(word_1)
                cos_sim_dict[word_1] = []
    
    handled_words = []
    for word_1 in list_1:
        for word_2 in list_2:
            if word_1 != word_2 and word_1 not in handled_words and word_2 not in handled_words:
                sim = find_lines_similarity(word_1, word_2)
                if sim > 0.95:
                    if word_1 in cos_sim_dict:
                        handled_words.append(word_2)
                        cos_sim_dict[word_1].append(word_2)
                       # overall_intersection[word_1_index] += '| ' + word_2
                    elif word_2 in cos_sim_dict:
                        handled_words.append(word_1)
                        cos_sim_dict[word_2].append(word_1)
                    else:
                        cos_sim_dict[word_1] = [word_2]
                        #new_pair = word_1 + " | " + word_2
                        #overall_intersection.append(new_pair)
    #print(cos_sim_dict)
    for first_word in cos_sim_dict.keys():
        intersect_element = first_word
        for similar_word in cos_sim_dict[first_word]:
            intersect_element += " | " + similar_word
        overall_intersection.append(intersect_element)
    return overall_intersection

fr_proc_1 = ['mère', 'maman', 'mère de famille', 'materner', 
             'vierge', 'mother', 'parent', 'femme', 'maternel', 'fil', 'dame', 'marie']
fr_proc_2 = ['mère', 'maman', 'putain', 'merde', 'quoi', 'enfoiré', 
             'est-ce', 'dieu', 'bordel', 'connard', 'enculé']
get_two_target_lang_lists_intersection(fr_proc_1, fr_proc_2)
    
                
            

['mère', 'maman']

In [12]:
def get_cross_translate(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_list = get_definitions(word_lang_1, lang_1, target_lang, print_output=True)
    lang_1_processed = preprocess_words(lang_1_to_target_words_list, lemmatizer_func)
    print(lang_1_processed)
    lang_2_to_target_words_list = get_definitions(word_lang_2, lang_2, target_lang, print_output=True)
    lang_2_processed = preprocess_words(lang_2_to_target_words_list, lemmatizer_func)
    print(lang_2_processed)
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_processed, lang_2_processed)
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate("warm", "английский", "согревать","русский", "французский", fr_lemmatize)
    

https://context.reverso.net/перевод/английский-французский/warm
['chaud', 'chaleureux', 'tiède', 'doux', 'bon', 'cordial', 'réchauffer', 'chauffer', 'échauffer', 'se réchauffer', 'tiédir', 'chaleur', 'accueil', 'chaleureusement', 'warm', 'accueillante', 'conviviale']
https://context.reverso.net/перевод/русский-французский/согревать
turn to alternative marks
['réchauffer', 'chaud']


['chaud', 'réchauffer | chauffer | échauffer']

In [13]:
def get_multilang_from_df(df):
    eng_word_list = []
    lang_1_word_list = []
    lang_2_word_list = []
    for df_ind in tqdm(range(len(df))):
        df_element = list(df.iloc[df_ind])
        eng_word = df_element[1]
        loc_1_word = df_element[2]
        print(eng_word, loc_1_word)
        sense_intersect = get_cross_translate(eng_word, "английский", loc_1_word,"русский", "французский", fr_lemmatize)
        print("sense_intersect", sense_intersect)
        for sense_intersect_element in sense_intersect:
            eng_word_list.append(eng_word)
            lang_1_word_list.append(loc_1_word)
            lang_2_word_list.append(sense_intersect_element)
            
        #break
        if df_ind % 10 == 0:
            save_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_barbos",
                         str(df_ind) + ".csv")
            data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =['eng_word', 'rus','fr'])
            data.to_csv(save_path)
    data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =['eng_word', 'rus','fr'])
    
    return data

df_din = get_multilang_from_df(df_en_ru_yandex)
df_din.head()


  0%|          | 0/676 [00:00<?, ?it/s]

for для
https://context.reverso.net/перевод/английский-французский/for
['pour', 'pendant', 'car', 'malgré', 'au titre de', 'à destination de', 'de', 'à', 'aux', 'par', 'permettant', 'dans', 'destiné', 'depuis', 'au', 'que', 'en', 'sur', 'durant', 'moyennant', 'puisque']
https://context.reverso.net/перевод/русский-французский/для


  0%|          | 1/676 [00:04<55:46,  4.96s/it]

['pour', 'qui', 'à', "d'", 'aux', 'afin', 'de', 'sur', 'dans', 'au', 'vue', 'en', 'par', "c'"]
sense_intersect ['pour', 'de', 'à', 'aux', 'par', 'dans', 'au', 'en', 'sur']
for ради
https://context.reverso.net/перевод/английский-французский/for
['pour', 'pendant', 'car', 'malgré', 'au titre de', 'à destination de', 'de', 'à', 'aux', 'par', 'permettant', 'dans', 'destiné', 'depuis', 'au', 'que', 'en', 'sur', 'durant', 'moyennant', 'puisque']
https://context.reverso.net/перевод/русский-французский/ради


  0%|          | 2/676 [00:11<59:31,  5.30s/it]

...

100%|█████████▉| 675/676 [57:49<00:04,  4.95s/it]

[]
sense_intersect ['no_equality']
read чтение
https://context.reverso.net/перевод/английский-французский/read
['lire', 'consulter', 'voir', 'relire', 'interpréter', 'déchiffrer', 'indiquer', 'bouquiner', 'pris connaissance', 'libellé', 'lecture', 'recevez', 'reçois', 'lisible', 'relever', 'read', 'en savoir', 'retrouvez']
https://context.reverso.net/перевод/русский-французский/чтение


100%|██████████| 676/676 [57:54<00:00,  4.97s/it]

['lecture', 'lire']
sense_intersect ['lire', 'lecture']


,eng_word,rus,fr
0,for,для,pour
1,for,для,de
2,for,для,à
3,for,для,aux
4,for,для,par


In [14]:
df_din.to_csv("cross_approach.csv")